# Exemple 2 : recalibrating crop simulation parameters based on optimization - North Cameroon Maize 1999-2022

The aim of this notebook is to show how to use the SARRA-Py package to perform multi-year yield simulations for Maize at the scale of northern Cameroon, and to compare results to a reference departemental yield database to back-feed a bayesian optimization algorithm to recalibrate some of the crop simulation parameters based on a criterion of maximization of correlation coefficient between simulated and observed yield. This is a rather advanced example of what can be done quite easily.

## Imports

In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm as tqdm
import xarray as xr
from sarra_py import *
import geopandas as gpd
import os
import urllib.request
import zipfile
import shutil
from contextlib import redirect_stdout, redirect_stderr
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
# if needed, we install the joblib package, which is used for parallel computing
# also, the package bayesian-optimization which is used for simulation parameter optimization
# (uncomment the following lines to trigger installation)
# !pip install joblib
# !pip install bayesian-optimization

# then we import these packages
from joblib import Parallel, delayed
from bayes_opt import BayesianOptimization

## 1. Loading maize yield observation data

Here, we will prepare the reference yield dataset that will be used for comparison against simulation results. 
As we use official departemental yield data, we first load the shapefile of departemental administrative boundaries with geopandas. Then, we load our statistics with pandas. Finally we associate the two objects based on departements names, by computing a text distance metric (Levenshtein).

In [3]:
# defining a Levenshtein distance helper function

def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

In [4]:
# load shapefile of departemental administrative boundaries
departements = gpd.read_file("../data/exemple_data/exemple_2/shp_division_north_Cmr/Départements_Nord_Extrême-Nord.shp")

# load official departmental maize yields
yields = pd.read_csv("../data/exemple_data/exemple_2/Nocamy_v1(North_Cameroon_Maize_Yields).csv", encoding="latin-1", sep=";")
yields["Yield"] = yields["Yield"].str.replace(",", ".").astype(float) * 1000 # fix decimal separator and convert units to kg/ha

# identify year min and year max from yield dataset
year_min_rdt = yields["Year"].min()
year_max_rdt = yields["Year"].max()
print("Year min:", year_min_rdt, "/ Year max:", year_max_rdt)

# match departments DIVISION name with yields Division name using string resemblance through Levenshtein distance
departements["Division"] = departements["DIVISION"].apply(lambda x: yields["Division"].iloc[np.argmin([levenshtein_distance(x, y) for y in yields["Division"]])])

# add annual yields columns to the departements dataframe based on the Division name matching
for year in range(year_min_rdt, year_max_rdt + 1):
    departements[str(year)] = departements["Division"].apply(lambda x: yields[(yields["Division"] == x) & (yields["Year"] == year)]["Yield"].values[0] if len(yields[(yields["Division"] == x) & (yields["Year"] == year)]) > 0 else np.nan)

Year min: 1999 / Year max: 2022


We obtain a geopandas dataframe that has departement names, geometries, and in columns the yearly yields from the reference dataset.

In [ ]:
departements.head(5)

## 2. Preparing climate and rainfall data

We download the already-prepared climate and rainfall data from the Zenodo repository. Climate data is from AgERA5, and rainfall is TAMSAT v3.1.

In [6]:
# downloading and unzipping the data from https://doi.org/10.5281/zenodo.14614281

# create a folder to store the data
os.makedirs('../data/exemple_data/exemple_2/', exist_ok=True)

# download preformatted data from Zenodo repository
url = 'https://zenodo.org/records/14614282/files/North_Cameroon_1990_2022.zip?download=1'
local_filename = '../data/exemple_data/exemple_2/North_Cameroon_1990_2022.zip' # store the downloaded file in the ../data/exemple_data/ folder
urllib.request.urlretrieve(url, local_filename)
print("File downloaded using urllib.")

# unzip data
path_to_zip_file = "../data/exemple_data/exemple_2/North_Cameroon_1990_2022.zip"
directory_to_extract_to = "../data/exemple_data/exemple_2/" # unzips the file in the same folder

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)
print("File unzipped.")


File downloaded using urllib.
File unzipped.


## 3. Preparing simulation parameters

We copy the provided example ITK and variety parameters in the right folder.

In [7]:
# we copy maize_cameroon_2020. yaml from the example folder to the ../data/params/itk/ folder 
shutil.copy("../data/exemple_data/exemple_2/itk_exemple_2.yaml", "../data/params/itk/itk_exemple_2.yaml")

# we copy maize_north_cameroon.yaml from the example folder to the ../data/params/variete/ folder
shutil.copy("../data/exemple_data/exemple_2/variety_exemple_2.yaml", "../data/params/variety/variety_exemple_2.yaml")

'../data/params/variety/variety_exemple_2.yaml'

## 4. Preparing and launching simulations

We first define a run_simulation function, which aim is to return the simulated yield for a given year and parameters. The parameters we want to optimize are phenological phase durations SDJBVP, SDJRPR, and SDJMatu1, and also a biomass partitioning parameter KRdtPotA. Being able to launch a full simulation in a function call will be useful later as we parallelize this action to speed up the whole process. The list of functions passed to launch simulations are identical to what is presented in Exemple 1, though it is way more condensed.

In [8]:
# Defining a simulation function to be able to parallelize
def run_simulation(year, KRdtPotA, SDJBVP, SDJRPR, SDJMatu1):

    # displaying the year
    print("===",year,"===")

    # hiding all the outputs of this code section
    with open(os.devnull, 'w') as devnull, redirect_stdout(devnull), redirect_stderr(devnull):

        # defining start date and simulatin duration ; simulation starts on April 1st and lasts until the end of the year
        date_start = datetime.date(year,4,1)
        duration = 365-(date_start-datetime.date(date_start.year,1,1)).days

        rainfall_data_path = "../data/exemple_data/exemple_2/TAMSAT_v3.1_north_cameroon_rfe_filled" # path to rainfall data
        climate_data_path = "../data/exemple_data/exemple_2/AgERA5_north_cameroon/" # path to climate data
        grid_width, grid_height = get_grid_size(rainfall_data_path, date_start, duration) # get grid size        
        base_data = xr.Dataset() # initialize empty xarray dataset
        base_data = load_TAMSAT_data(base_data, rainfall_data_path, date_start, duration) # load rainfall data
        base_data = load_AgERA5_data(base_data, climate_data_path, date_start, duration) # load climate data        
        base_data = load_iSDA_soil_data_alternate(base_data, grid_width, grid_height) # load soil parameters
        base_data = calc_day_length_raster_fast(base_data, date_start, duration) # compute day length raster

        # load variety, cropping system and soil parameters
        file_paramVariete = "maize_north_cameroon.yaml"
        file_paramITK = "maize_cameroon_2020.yaml"
        file_paramTypeSol = "USA_iowa_V42.yaml"
        paramVariete, paramITK, paramTypeSol = load_YAML_parameters(file_paramVariete, file_paramITK, file_paramTypeSol)

        # updating parameters
        paramITK["DateSemis"] = date_start # forcing the sowing date to be the start date
        paramVariete["KRdtPotA"] = KRdtPotA # forcing the optimization parameters
        paramVariete["SDJBVP"] = SDJBVP
        paramVariete["SDJRPR"] = SDJRPR
        paramVariete["SDJMatu1"] = SDJMatu1

        # creating simulation xarray dataset by copying the base data ; initializing all the necessary variables
        data = base_data.copy() 
        data = initialize_simulation(data, grid_width, grid_height, duration, paramVariete, paramITK, date_start)
        data = initialize_default_irrigation(data)
        data = calculate_once_daily_thermal_time(data, paramVariete)

    data = run_model(paramVariete, paramITK, paramTypeSol, data, duration) # running the model
    rdt = data["rdt"][-1,:,:] # extracting the simulated yield (last timepoint)

    # freeing memory
    del base_data
    del data

    return rdt

Then, we define a black box objective function function that aims at launching crop simulations in parallel, collect multi-year simulated yields, compute the mean of simulated yields inside each department limits, compare it to the official yields and compute a metric - here a Pearson correlation, as SARRA-Py is notoriously known to overestimate yields. However this code also contains calculation of R2 and RMSE metrics if you want to test it.

In [9]:
# defining the objective function
def run_sim_and_calc_score(KRdtPotA, SDJBVP, SDJRPR, SDJMatu1):

    year_min, year_max = 1999, 2022 # define the interval of years on which to run the simulations
    parallel_jobs = 4 # if you have lots of RAM increase the number of parallel jobs
    results = Parallel(n_jobs=parallel_jobs)(delayed(run_simulation)(year, KRdtPotA, SDJBVP, SDJRPR, SDJMatu1) for year in range(year_min, year_max+1)) # parallel run the simulations
    rdt = xr.concat(results, dim="year") # assemble the results in a single xarray dataset

    # loop over the years and departments to calculate the mean and median of the simulated yield for each of them
    results = pd.DataFrame() # initialize results dataframe

    # loop over the years and departments
    for year in tqdm(range(np.max([year_min_rdt, year_min]), np.min([year_max_rdt, year_max]) + 1)):
        for i in range(len(departements)):
            index_mean = np.mean(rdt[year_max-year,:,:].rio.clip([departements.iloc[i].geometry])).values # compute departmental mean yield
            index_median = (rdt[year_max-year,:,:].rio.clip([departements.iloc[i].geometry])).median().values # compute departmental median yield
            results = pd.concat([results, pd.DataFrame({"year": year, "departement": departements.loc[i, "Division"], "rendement": departements.loc[i, str(year)], "rdt_sim_mean": index_mean, "rdt_sim_median": index_median}, index=[0])], ignore_index=True) # append results to the dataframe

    # calculate metrics
    r2 = r2_score(results.dropna()["rendement"], results.dropna()["rdt_sim_mean"])
    rmse = np.sqrt(mean_squared_error(results.dropna()["rendement"], results.dropna()["rdt_sim_mean"]))
    pearson = np.corrcoef(results.dropna()["rendement"], results.dropna()["rdt_sim_mean"])[0,1]
    print("R2:", r2, "/ RMSE:", rmse, "/ Pearson:", pearson)


    return pearson

Finally, we define the boundaries in which we look for a global optimum for each variable, and launch the optimization.

In [ ]:
# setting boundaries
# KrdtPotA: between 0.35 and 1.2
# SDJBVP: between 200 and 500 
# SDJRPR: between 360 and 630
# SDJMatu1: between 370 and 900

pbounds = {'KRdtPotA': (0.35, 1.2),
           'SDJBVP': (200, 500),
           'SDJRPR': (360, 630),
           'SDJMatu1': (370, 900)}

# instantiating the optimizer
optimizer = BayesianOptimization(
    f=run_sim_and_calc_score,
    pbounds=pbounds,
    random_state=1,
)

# running the optimization, for threee initial points and ten iterations
# (we can increase the number of initial points and iterations to improve the optimization)
optimizer.maximize(
    init_points=3,
    n_iter=10,
)

In [ ]:
# displaying the best identified parameters
print(optimizer.max)